In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("weather.csv", index_col="DATE")
df.head()

,STATION,NAME,ACMH,ACSH,AWND,FMTM,FRGT,GAHT,PGTM,PRCP,...,WT14,WT15,WT16,WT17,WT18,WT19,WT21,WT22,WV03,WV20
DATE,,,,,,,,,,,,,,,,,,,,,
1960-01-01,USW00093821,"LOUISVILLE INTERNATIONAL AIRPORT, KY US",NaN,NaN,NaN,NaN,NaN,-218.5,NaN,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1960-01-02,USW00093821,"LOUISVILLE INTERNATIONAL AIRPORT, KY US",NaN,NaN,NaN,NaN,NaN,-233.9,NaN,0.38,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1960-01-03,USW00093821,"LOUISVILLE INTERNATIONAL AIRPORT, KY US",NaN,NaN,NaN,NaN,NaN,-244.9,NaN,0.01,...,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1960-01-04,USW00093821,"LOUISVILLE INTERNATIONAL AIRPORT, KY US",NaN,NaN,NaN,NaN,NaN,-281.9,NaN,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1960-01-05,USW00093821,"LOUISVILLE INTERNATIONAL AIRPORT, KY US",NaN,NaN,NaN,NaN,NaN,-301.2,NaN,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Count null values, % by column 
df.apply(pd.isnull).sum()/df.shape[0]
df.head()

,STATION,NAME,ACMH,ACSH,AWND,FMTM,FRGT,GAHT,PGTM,PRCP,...,WT14,WT15,WT16,WT17,WT18,WT19,WT21,WT22,WV03,WV20
DATE,,,,,,,,,,,,,,,,,,,,,
1960-01-01,USW00093821,"LOUISVILLE INTERNATIONAL AIRPORT, KY US",NaN,NaN,NaN,NaN,NaN,-218.5,NaN,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1960-01-02,USW00093821,"LOUISVILLE INTERNATIONAL AIRPORT, KY US",NaN,NaN,NaN,NaN,NaN,-233.9,NaN,0.38,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1960-01-03,USW00093821,"LOUISVILLE INTERNATIONAL AIRPORT, KY US",NaN,NaN,NaN,NaN,NaN,-244.9,NaN,0.01,...,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1960-01-04,USW00093821,"LOUISVILLE INTERNATIONAL AIRPORT, KY US",NaN,NaN,NaN,NaN,NaN,-281.9,NaN,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1960-01-05,USW00093821,"LOUISVILLE INTERNATIONAL AIRPORT, KY US",NaN,NaN,NaN,NaN,NaN,-301.2,NaN,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Documentation: keep five core measurements:
# 1. Precipitation (PRCP)
# 2. Snowfall (SNOW)
# 3. Snow depth (SNWD)
# 4. Max temp (TMAX)
# 5. Min temp (TMIN)

core = df[["PRCP", "SNOW", "SNWD", "TMAX", "TMIN"]].copy()
core.head()

,PRCP,SNOW,SNWD,TMAX,TMIN
DATE,,,,,
1960-01-01,0.00,0.0,0.0,41.0,27.0
1960-01-02,0.38,0.0,0.0,48.0,37.0
1960-01-03,0.01,0.0,0.0,43.0,29.0
1960-01-04,0.00,0.0,0.0,33.0,24.0
1960-01-05,0.00,0.0,0.0,33.0,26.0


In [5]:
# Rename columns
core.columns = ["precip", "snow", "snow_depth", "temp_max", "temp_min"]